In [1]:
import pandas as pd
import numpy as np
import janitor as jn

In [2]:
df1 = pd.DataFrame({"id": [1, 1, 1, 2, 2, 3], "value_1": [2, 5, 7, 1, 3, 4]})
df2 = pd.DataFrame(
    {
        "id": [1, 1, 1, 1, 2, 2, 2, 3],
        "value_2A": [0, 3, 7, 12, 0, 2, 3, 1],
        "value_2B": [1, 5, 9, 15, 1, 4, 6, 3],
    }
) 

In [3]:
df1

,id,value_1
0,1,2
1,1,5
2,1,7
3,2,1
4,2,3
5,3,4


In [4]:
df2

,id,value_2A,value_2B
0,1,0,1
1,1,3,5
2,1,7,9
3,1,12,15
4,2,0,1
5,2,2,4
6,2,3,6
7,3,1,3


In [5]:
A=df1.sample(len(df1)).reset_index(drop=True)
B = df2.sample(len(df2)).reset_index(drop=True)

In [6]:
df1.conditional_join(
    df2,
        ("value_1", "value_2A", ">="),
       ("id", "id", "=="),
    
     ("value_1", "value_2B", "<="),
    use_numba=True,
)

([array([1, 3, 4, 0, 1, 2]),
  array([0, 0, 0, 1, 1, 2]),
  array([2, 1, 0, 3, 1, 1])],
 [array([0, 0, 1, 2, 3, 3, 4]),
  array([1, 0, 2, 1, 0, 1, 0]),
  array([3, 3, 3, 2, 1, 1, 0])])

In [7]:
A,B

(   id  value_1
 0   1        2
 1   1        5
 2   2        3
 3   3        4
 4   2        1
 5   1        7,
    id  value_2A  value_2B
 0   2         3         6
 1   2         2         4
 2   1         7         9
 3   2         0         1
 4   1        12        15
 5   1         0         1
 6   1         3         5
 7   3         1         3)

In [8]:
df1,df2

(   id  value_1
 0   1        2
 1   1        5
 2   1        7
 3   2        1
 4   2        3
 5   3        4,
    id  value_2A  value_2B
 0   1         0         1
 1   1         3         5
 2   1         7         9
 3   1        12        15
 4   2         0         1
 5   2         2         4
 6   2         3         6
 7   3         1         3)

In [9]:
df1.conditional_join(
    df2,
     ("id", "id", "=="),
    ("value_1", "value_2A", ">="),
    # ('id','id','<='),
    ("value_1", "value_2B", "<="),
    use_numba=False,
)

(array([1, 2, 3, 4, 4]), array([1, 2, 4, 5, 6]))

In [10]:
print 'i'

SyntaxError: Missing parentheses in call to 'print'. Did you mean print('i')? (414025362.py, line 1)

In [11]:
url = "https://raw.githubusercontent.com/samukweku/data-wrangling-blog/master/notebooks/Data_files/flights.csv"
flights = pd.read_csv(url, sep = '\t', names=['orig','dest','orig_time', 'dest_time'], parse_dates  = ['orig_time', 'dest_time'])
flights = flights.factorize_columns(['orig','dest']).iloc[:, 2:]
flights.columns = [ent.split("_")[0] if ent.endswith('enc') else ent for ent in flights]
flights.columns = ['takeoff','landing','orig','dest']
flights = flights.assign(start=flights.landing+pd.Timedelta(minutes=45), end=flights.landing+pd.Timedelta(hours=3))
flights.head()

,takeoff,landing,orig,dest,start,end
0,2021-11-27 07:15:00,2021-11-27 08:55:00,0,0,2021-11-27 09:40:00,2021-11-27 11:55:00
1,2021-11-27 20:05:00,2021-11-28 00:50:00,1,1,2021-11-28 01:35:00,2021-11-28 03:50:00
2,2021-11-27 21:00:00,2021-11-27 21:35:00,2,2,2021-11-27 22:20:00,2021-11-28 00:35:00
3,2021-11-27 21:15:00,2021-11-27 22:25:00,0,2,2021-11-27 23:10:00,2021-11-28 01:25:00
4,2021-11-26 11:40:00,2021-11-26 14:45:00,3,3,2021-11-26 15:30:00,2021-11-26 17:45:00


In [ ]:
flights.tail()

,takeoff,landing,orig,dest,start,end
127421,2022-01-29 13:45:00,2022-01-29 15:35:00,18,27,2022-01-29 16:20:00,2022-01-29 18:35:00
127422,2022-01-29 11:20:00,2022-01-29 12:20:00,27,2,2022-01-29 13:05:00,2022-01-29 15:20:00
127423,2022-01-29 16:00:00,2022-01-29 20:15:00,94,143,2022-01-29 21:00:00,2022-01-29 23:15:00
127424,2022-01-29 09:20:00,2022-01-29 10:55:00,27,27,2022-01-29 11:40:00,2022-01-29 13:55:00
127425,2022-01-29 11:35:00,2022-01-29 14:55:00,94,119,2022-01-29 15:40:00,2022-01-29 17:55:00


In [12]:
(flights
 .conditional_join(
     flights, 
     ('end', 'takeoff' ,'>='), 
     ('start', 'takeoff', '<='), 
    #   ('orig','orig','!='), 
     ('dest', 'orig', '=='), 
    df_columns = ['start', 'end', 'dest'],
     right_columns = ['takeoff', 'orig'],
     use_numba=True)
)

([array([   25,   165,   157, ..., 10975, 10863, 10911]),
  array([  0,   1,   2, ..., 143,  27, 119]),
  array([17287, 17173, 17173, ...,  8012,  8120,  8072])],
 [array([    0,     0,     0, ..., 20459, 20459, 20460]),
  array([ 47,  15, 130, ...,   8,  22,  41]),
  array([17311, 17311, 17311, ...,     2,     1,     0])])

In [ ]:
%%timeit
(flights
 .conditional_join(
     flights, 
     ('end', 'takeoff' ,'>='), 
     ('start', 'takeoff', '<='), 
    #   ('orig','orig','!='), 
    #  ('dest', 'orig', '=='), 
    df_columns = ['start', 'end', 'dest'],
     right_columns = ['takeoff', 'orig'],
     use_numba=False)
)

122 ms ± 1.44 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%%timeit
(flights
 .conditional_join(
     flights, 
     ('end', 'takeoff' ,'>='), 
     ('start', 'takeoff', '<='), 
    #   ('orig','orig','!='), 
     ('dest', 'orig', '=='), 
    df_columns = ['start', 'end', 'dest'],
     right_columns = ['takeoff', 'orig'],
     use_numba=True)
)

51.4 ms ± 5.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
(flights
 .conditional_join(
     flights, 
     ('end', 'takeoff' ,'>='), 
     ('start', 'takeoff', '<='), 
    #   ('orig','orig','!='), 
     ('dest', 'orig', '=='), 
    df_columns = ['start', 'end', 'dest'],
     right_columns = ['takeoff', 'orig'],
     use_numba=False)
)[0].size


85001

In [ ]:
%%timeit
(flights
 .conditional_join(
     flights, 
     ('end', 'takeoff' ,'>='), 
     ('start', 'takeoff', '<='), 
      ('orig','orig','!='), 
     ('dest', 'orig', '=='), 
    df_columns = ['start', 'end', 'dest'],
     right_columns = ['takeoff', 'orig'],
     use_numba=False)
)

2.74 s ± 126 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
(flights
 .conditional_join(
     flights, 
     ('end', 'takeoff' ,'>='), 
     ('start', 'takeoff', '<='), 
    #  ('orig','orig','!='), 
    # ('dest', 'orig', '=='), 
    df_columns = ['start', 'end', 'dest'],
     right_columns = ['takeoff', 'orig'],
     use_numba=True)
)

(array([123627, 123501, 123501, ...,  42298,  42298,  42298]),
 array([123853, 123853, 123719, ...,  42263,     54,  42285]))

In [ ]:
%%timeit
(flights
 .conditional_join(
     flights, 
     ('end', 'takeoff' ,'>='), 
     ('start', 'takeoff', '<='), 
      ('orig','orig','!='), 
     ('dest', 'orig', '=='), 
    df_columns = ['start', 'end', 'dest'],
     right_columns = ['takeoff', 'orig'],
     force=True)
)

In [ ]:
%timeit flights.loc[:,['orig','takeoff']].sort_values(['orig','takeoff'])

In [ ]:
%%timeit 
flights.takeoff.sort_values(), flights.orig.sort_values()

In [ ]:
df = {
    "A": [0, 0],
    "B": [-1.0, 0.0],
    "E": pd.to_datetime(["1970-01-01", "1970-01-01"]),
}


right = {
    "Integers": [0, 1, 1],
    "Numeric": [0.0, 1.0, 0.0],
    "Dates": pd.to_datetime(
        [
            "1970-01-01 00:00:00.000000000",
            "1969-12-31 23:59:59.999999999",
            "1969-12-31 23:59:59.999999999",
        ]
    ),
    "Dates_Right": pd.to_datetime(
        [
            "1970-01-01",
            "1970-01-01",
            "1970-01-01",
        ]
    ),
}


right = pd.DataFrame(right)

df = pd.DataFrame(df)

In [ ]:
df

In [ ]:
right

In [ ]:
filters = ["A", "B", "E", "Integers", "Numeric", "Dates"]
df = df.assign(A=df["A"].astype("Int64"))
right = right.assign(Integers=right["Integers"].astype(pd.Int64Dtype()))

In [ ]:
expected = df[["A", "B", "E"]].merge(
            right[["Integers", "Numeric", "Dates"]], how="cross"
        )

expected

In [ ]:
expected.loc[
            # expected.A.ne(expected.Integers)
             expected.B.lt(expected.Numeric)

            
            & expected.E.ge(expected.Dates),
        ]

In [ ]:
pd.__version__

In [ ]:
(df[["A", "B", "E"]]
            .conditional_join(
                right[["Integers", "Numeric", "Dates"]],
                ("E", "Dates", ">="),
                  ("A", "Integers", "!="),
                ("B", "Numeric", "<"),
                how="inner",
                use_numba=True,
            ))

In [ ]:
A =(
    df[["E"]].conditional_join(
        right[["Dates", "Dates_Right"]],
        ("E", "Dates", ">="),
        ("E", "Dates_Right", "<="),
        how="inner",
        use_numba=True,
    )
)

A

In [ ]:
B = (
    df[["E"]].conditional_join(
        right[["Dates", "Dates_Right"]],
        ("E", "Dates", ">="),
        ("E", "Dates_Right", "<="),
        how="inner",
        use_numba=False,
    )
)

B

In [ ]:
A.sort_values(["E", "Dates", "Dates_Right"], ignore_index=True)

In [ ]:
C.sort_values(["E", "Dates", "Dates_Right"], ignore_index=True)

In [ ]:
B.sort_values(["E", "Dates", "Dates_Right"], ignore_index=True)

In [ ]:
C = (
    df[["E"]]
    .merge(right[["Dates", "Dates_Right"]], how="cross")
    .loc[lambda df: df.E.between(df.Dates, df.Dates_Right, inclusive="both")]
)

C

In [ ]:
C.sort_values(["E", "Dates", "Dates_Right"], ignore_index=True)

In [ ]:
df.dtypes, right.dtypes

In [ ]:
(
    df[["A", "E"]].conditional_join(
        right[["Integers", "Dates"]],
        ("E", "Dates", "<"),
        ("A", "Integers", ">="),
        how="inner",
        use_numba=True,
        sort_by_appearance=False,
    )
)

In [ ]:
columns = ["A", "E", "Integers", "Dates"]
(
    df.merge(
        right,
        how="cross",
    )
    .loc[lambda df: df.A.ge(df.Integers) & df.E.lt(df.Dates), columns]
    .sort_values(columns, ignore_index=True)
)

In [ ]:
(
    df[["A", "E"]].conditional_join(
        right[["Integers", "Dates"]],
        ("E", "Dates", "<"),
        ("A", "Integers", ">="),
        how="inner",
        use_numba=False,
        sort_by_appearance=False,
    )
)

In [ ]:
print 'j'

In [ ]:
pd.__version__

In [ ]:
df1 = pd.DataFrame({"id": [1, 1, 1, 2, 2, 3], "value_1": [2, 5, 7, 1, 3, 4]})
df2 = pd.DataFrame(
    {
        "id": [1, 1, 1, 1, 2, 2, 2, 3],
        "value_2A": [0, 3, 7, 12, 0, 2, 3, 1],
        "value_2B": [1, 5, 9, 15, 1, 4, 6, 3],
    }
)

In [ ]:
df1 = pd.concat([df1] * 20000)
df2 = pd.concat([df2] * 300)

df1.conditional_join(
    df2,
    ("value_1", "value_2A", ">"),
    ("value_1", "value_2B", "<"),
    use_numba=True,
)

In [ ]:
df1.conditional_join(
    df2,
    ("value_1", "value_2A", ">"),
    ("value_1", "value_2B", "<"),
    use_numba=False,
)

In [ ]:
%timeit df1.conditional_join(df2, ('value_1','value_2A','>'), ('value_1','value_2B','<'),use_numba=True)

In [ ]:
# 32.6ms   30_000_000

In [ ]:
df1

In [ ]:
# events = pd.read_csv(
#     "/home/sam/Downloads/results.csv", parse_dates=["start", "end"]
# ).iloc[:, 1:]
# events.head()

In [ ]:
events = pd.read_csv(
    "/Users/samuel.oranyeli/Downloads/results.csv",
    parse_dates=["start", "end"],
).iloc[:, 1:]

In [ ]:
a = 1
a.max()

In [ ]:
A = events.conditional_join(
    events,
    ("start", "end", "<="),
    ("end", "start", ">="),
    # ('id', 'id', '!='),
    # ('audience','audience','>'),
    use_numba=True,
    df_columns=["id", "start", "end"],
    right_columns=["id", "start", "end"],
)

A

In [ ]:
B = events.conditional_join(
    events,
    ("start", "end", "<="),
    ("end", "start", ">="),
    # ('id', 'id', '!='),
    # ('audience','audience','>'),
    use_numba=False,
    df_columns=["id", "start", "end"],
    right_columns=["id", "start", "end"],
)

In [ ]:
A.sort_values(A.columns.tolist(), ignore_index=True).equals(
    B.sort_values(A.columns.tolist(), ignore_index=True)
)

In [ ]:
%%timeit
(
    events.conditional_join(
        events,
        ("start", "end", "<="),
        ("end", "start", ">="),
        # ('id', 'id', '!='),
        # ('audience','audience','>'),
        use_numba=True,
        df_columns=["id", "start", "end"],
        right_columns=["id", "start", "end"],
    )
)

In [ ]:
%%timeit
(
    events.conditional_join(
        events,
        ("start", "end", "<="),
        ("end", "start", ">="),
        # ('id', 'id', '!='),
        # ('audience','audience','>'),
        use_numba=False,
        df_columns=["id", "start", "end"],
        right_columns=["id", "start", "end"],
    )
)

In [ ]:
C = dict(
    key=[f"c{num}" for num in range(1, 8)],
    vol=[35, 15, 5, 35, 18, 90, 17],
    profit=[45, 35, 55, 12, 15, 55, 11],
    unitsSold=[15, 10, 30, 10, 15, 80, 2],
    keyy=range(1, 8),
)
C = pd.DataFrame(C)


D = dict(
    key=[f"d{num}" for num in range(1, 9)],
    vol=[20, 50, 15, 16, 40, 20, 40, 2],
    profit=[30, 10, 12, 52, 35, 20, 30, 57],
    unitsSold=[20, 35, 10, 12, 40, 30, 5, 15],
    keyy=range(8, 0, -1),
)
D = pd.DataFrame(D)

In [ ]:
C = pd.concat([C] * 400)
D = pd.concat([D] * 400)

In [ ]:
C.conditional_join(
    D,
    ("profit", "profit", ">="),
     ('keyy','keyy','<='),
      ('unitsSold','unitsSold','>='),
    # ("vol", "vol", "<="),
      ('vol','profit','>'),
    use_numba=True,
)

In [ ]:
C.conditional_join(
    D,
    ("profit", "profit", ">="),
     ('keyy','keyy','<='),
      ('unitsSold','unitsSold','>='),
    # ("vol", "vol", "<="),
      ('vol','profit','>'),
    use_numba=False,
)

In [ ]:
%%timeit
out = C.conditional_join(
    D,
    ("profit", "profit", ">="),
     ('keyy','keyy','<='),
      ('unitsSold','unitsSold','>='),
    # ("vol", "vol", "<="),
      ('vol','profit','>'),
    use_numba=True,
)

In [ ]:
%%timeit
C.conditional_join(
    D,
    ("profit", "profit", ">="),
     ('keyy','keyy','<='),
      ('unitsSold','unitsSold','>='),
    # ("vol", "vol", "<="),
      ('vol','profit','>'),
    use_numba=True,
)


In [ ]:
%%timeit
out = C.conditional_join(
    D,
    ("profit", "profit", ">="),
     ('keyy','keyy','<='),
      ('unitsSold','unitsSold','>='),
    # ("vol", "vol", "<="),
      ('vol','profit','>'),
    use_numba=False,
)



In [ ]:
C.conditional_join(
    D,
    ("profit", "profit", ">="),
     ('keyy','keyy','<='),
      ('unitsSold','unitsSold','>='),
    # ("vol", "vol", "<="),
      ('vol','profit','>'),
    use_numba=False,
)[0].size


In [ ]:
C.conditional_join(
    D,
    ("profit", "profit", ">="),
     ('keyy','keyy','<='),
      ('unitsSold','unitsSold','>='),
    # ("vol", "vol", "<="),
      ('vol','profit','>'),
    use_numba=False,
)

In [ ]:
(
    C.conditional_join(
        D,
        ("profit", "profit", ">="),
        # ('keyy','keyy','=='),
          # ('unitsSold','unitsSold','>='),
        ("vol", "vol", "<="),
        # ('vol','profit','>'),
        use_numba=False,
    )
)

In [ ]:
%%timeit
(
    C.conditional_join(
        D,
        ("profit", "profit", ">="),
        # ('keyy','keyy','=='),
        #  ('unitsSold','unitsSold','>='),
        ("vol", "vol", "<="),
        # ('vol','profit','>'),
        use_numba=True,
    )
)

In [ ]:
%%timeit
(
    C.conditional_join(
        D,
        ("profit", "profit", ">="),
        # ('keyy','keyy','=='),
        #  ('unitsSold','unitsSold','>='),
        ("vol", "vol", "<="),
        # ('vol','profit','>'),
        use_numba=False,
    )
)

In [ ]:
print 'i'

In [ ]:
C = pd.concat([C] * 400)
D = pd.concat([D] * 400)

In [ ]:
(
    C.conditional_join(
        D,
        ("profit", "profit", ">="),
        # ('keyy','keyy','=='),
        # ('unitsSold','unitsSold','>='),
        ("vol", "vol", "<="),
        # ('vol','profit','>'),
        use_numba=True,
    )
)

In [ ]:
%%timeit
(
    C.conditional_join(
        D,
        ("profit", "profit", ">="),
        # ('keyy','keyy','=='),
        # ('unitsSold','unitsSold','>='),
        ("vol", "vol", "<="),
        # ('vol','profit','>'),
        use_numba=True,
    )
)

In [ ]:
%%timeit
(
    C.conditional_join(
        D,
        ("profit", "profit", ">="),
        # ('keyy','keyy','=='),
        # ('unitsSold','unitsSold','>='),
        ("vol", "vol", "<="),
        # ('vol','profit','>'),
        use_numba=False,
    )
)

In [ ]:
(
    C.conditional_join(
        D,
        ("profit", "profit", ">="),
        # ('keyy','keyy','=='),
        # ('unitsSold','unitsSold','>='),
        ("vol", "vol", "<="),
        # ('vol','profit','>'),
        use_numba=True,
    )
)[0].size

In [ ]:
np.asarray(1), np.asanyarray(1)

In [ ]:
(
    C.conditional_join(
        D,
        ("profit", "profit", ">="),
        # ('keyy','keyy','=='),
        # ('unitsSold','unitsSold','>='),
        ("vol", "vol", "<="),
        # ('vol','profit','>'),
        use_numba=False,
    )
)[0].size

In [ ]:
from string import ascii_lowercase

np.random.seed(1)
# n = 10000000; k = 20000
n = 50
k = 10
mapp = {k: v for v, k in enumerate(ascii_lowercase)}
idx1 = np.random.randint(0, high=2_00, size=n)
idx2 = np.random.randint(0, high=3_00, size=n)

d1 = dict(
    x=np.random.choice(list(ascii_lowercase[:5]), size=n),
    start=np.minimum(idx1, idx2),
    end=np.maximum(idx1, idx2),
)


d2 = dict(
    x=np.random.choice(list(ascii_lowercase[:15]), size=k),
    pos1=np.random.randint(low=60, high=151, size=k),
)

d1 = pd.DataFrame(d1)
d2 = pd.DataFrame(d2)
d1 = d1.assign(xx=lambda df: df.x.map(mapp))
d2 = d2.assign(xx=lambda df: df.x.map(mapp))

In [ ]:
d1

,x,start,end,xx
0,e,37,76,4
1,b,26,140,1
2,a,72,109,0
3,e,137,297,4
4,c,15,133,2
5,a,64,79,0
6,c,192,196,2
7,e,25,144,4
8,b,129,226,1
9,b,71,282,1


In [ ]:
d2

,x,pos1,xx
0,o,69,14
1,n,126,13
2,e,131,4
3,e,113,4
4,l,129,11
5,g,96,6
6,n,81,13
7,i,100,8
8,l,137,11
9,i,109,8


In [ ]:
d2.conditional_join(
    d1,
    ("pos1", "start", ">"),
    ("pos1", "end", "<"),
     ('xx', 'xx', '=='),
    use_numba=True,
)

[array([[0, 1, 4],
       [0, 4, 3]]),
 array([[0, 8, 0],
       [0, 8, 5],
       [0, 2, 5],
       [0, 7, 5],
       [0, 8, 5],
       [0, 8, 5]]),
 array([0, 0]),
 array([6, 6]),
 array([8, 8])]
0 2 1 1 4 5 2 1
0 7 1 1 4 5 3 1
0 8 4 1 4 5 1 1
0 8 4 1 4 5 4 1
0 8 4 1 4 5 5 1
1 7 1 4 3 5 3 1
1 8 4 4 3 5 1 1
1 8 4 4 3 5 4 1
1 8 4 4 3 5 5 1


(array([[1, 4],
        [4, 3]]),
 array([[8, 0],
        [8, 5],
        [2, 5],
        [7, 5],
        [8, 5],
        [8, 5]]),
 array([0, 0]),
 array([6, 6]))

In [ ]:
d2.conditional_join(
    d1,
    ("pos1", "start", ">"),
    ("pos1", "end", "<"),
     ('xx', 'xx', '=='),
    # use_numba=True,
)

(array([2, 2, 2, 2, 2, 3, 3, 3, 3]),
 array([ 7, 12, 13, 15, 40,  7, 12, 13, 40]))

In [ ]:
%%timeit
bla = d2.conditional_join(
    d1,
    ("pos1", "start", ">"),
    ("pos1", "end", "<"),
     ('xx', 'xx', '=='),
    # use_numba=True,
)

512 µs ± 19.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
%%timeit
bla = d2.conditional_join(
    d1,
    ("pos1", "start", ">"),
    ("pos1", "end", "<"),
     ('xx', 'xx', '=='),
    use_numba=True,
)

[array([[0, 1, 4],
       [0, 4, 3]]),
 array([[0, 8, 0],
       [0, 8, 5],
       [0, 2, 5],
       [0, 7, 5],
       [0, 8, 5],
       [0, 8, 5]]),
 array([0, 0]),
 array([6, 6]),
 array([8, 8])]
[array([[0, 1, 4],
       [0, 4, 3]]),
 array([[0, 8, 0],
       [0, 8, 5],
       [0, 2, 5],
       [0, 7, 5],
       [0, 8, 5],
       [0, 8, 5]]),
 array([0, 0]),
 array([6, 6]),
 array([8, 8])]
[array([[0, 1, 4],
       [0, 4, 3]]),
 array([[0, 8, 0],
       [0, 8, 5],
       [0, 2, 5],
       [0, 7, 5],
       [0, 8, 5],
       [0, 8, 5]]),
 array([0, 0]),
 array([6, 6]),
 array([8, 8])]
[array([[0, 1, 4],
       [0, 4, 3]]),
 array([[0, 8, 0],
       [0, 8, 5],
       [0, 2, 5],
       [0, 7, 5],
       [0, 8, 5],
       [0, 8, 5]]),
 array([0, 0]),
 array([6, 6]),
 array([8, 8])]
[array([[0, 1, 4],
       [0, 4, 3]]),
 array([[0, 8, 0],
       [0, 8, 5],
       [0, 2, 5],
       [0, 7, 5],
       [0, 8, 5],
       [0, 8, 5]]),
 array([0, 0]),
 array([6, 6]),
 array([8, 8])]
[array([[0

In [ ]:
%%timeit
bla = d2.conditional_join(
    d1,
    ("pos1", "start", ">"),
    ("pos1", "end", "<"),
     ('xx', 'xx', '=='),
    use_numba=False,
)

1.78 s ± 154 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit
bla = d2.conditional_join(
    d1,
    ("pos1", "start", ">"),
    ("pos1", "end", "<"),
    # ('xx', 'xx', '=='),
    use_numba=False,
)

In [ ]:
print 'jarr'

In [ ]:
employees = pd.read_csv("/Users/samuel.oranyeli/employees.csv")

In [ ]:
%%timeit
(
    employees.conditional_join(
        employees, ("salary", "salary", "<"), ("tax", "tax", ">"), use_numba=True
    )
)

In [ ]:
%%timeit
(
    employees.conditional_join(
        employees, ("salary", "salary", "<"), ("tax", "tax", ">"), use_numba=False
    )
)